In [12]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from datetime import datetime
from sortedcontainers import SortedList

# load in the data
import os
if not os.path.exists('user2movie.json') or \
   not os.path.exists('movie2user.json') or \
   not os.path.exists('usermovie2rating.json') or \
   not os.path.exists('usermovie2rating_test.json'):
   import preprocess2dict


with open('user2movie.json', 'rb') as f:
  user2movie = pickle.load(f)

with open('movie2user.json', 'rb') as f:
  movie2user = pickle.load(f)

with open('usermovie2rating.json', 'rb') as f:
  usermovie2rating = pickle.load(f)

with open('usermovie2rating_test.json', 'rb') as f:
  usermovie2rating_test = pickle.load(f)

In [13]:
import numpy as np

N = np.max(list(user2movie.keys())) + 1
m1 = np.max(list(movie2user.keys()))
m2 = np.max([m for (u, m), r in usermovie2rating_test.items()])
M = max(m1, m2) + 1
print("N:", N, "M:", M)

if N > 10000:
    print("N =", N, "are you sure you want to continue?")
    print("Comment out these lines if so...")
    exit()

K = 25
limit = 5
neighbors = []
averages = []
deviations = []

for i in range(N):
    movies_i = user2movie[i]
    movies_i_set = set(movies_i)

    ratings_i = {movie: usermovie2rating[(i, movie)] for movie in movies_i}
    avg_i = np.mean(list(ratings_i.values()))
    dev_i = {movie: (rating - avg_i) for movie, rating in ratings_i.items()}
    dev_i_values = np.array(list(dev_i.values()))
    sigma_i = np.sqrt(dev_i_values.dot(dev_i_values))

    averages.append(avg_i)
    deviations.append(dev_i)


N: 10000 M: 2000


In [14]:
from sortedcontainers import SortedList
import numpy as np

sl = SortedList()

for j in range(N):
    if j != i:
        movies_j = user2movie[j]
        movies_j_set = set(movies_j)
        common_movies = movies_i_set & movies_j_set
        if len(common_movies) > limit:
            ratings_j = {movie: usermovie2rating[(j, movie)] for movie in movies_j}
            avg_j = np.mean(list(ratings_j.values()))
            dev_j = {movie: (rating - avg_j) for movie, rating in ratings_j.items()}
            dev_j_values = np.array(list(dev_j.values()))
            sigma_j = np.sqrt(dev_j_values.dot(dev_j_values))

            numerator = sum(dev_i[m] * dev_j[m] for m in common_movies)
            w_ij = numerator / (sigma_i * sigma_j)

            sl.add((-w_ij, j))
            if len(sl) > K:
                del sl[-1]

neighbors.append(sl)

if i % 1 == 0:
    print(i)


9999


In [15]:
def predict(i, m):
  # calculate the weighted sum of deviations
  numerator = 0
  denominator = 0
  for neg_w, j in neighbors[i]:
    # remember, the weight is stored as its negative
    # so the negative of the negative weight is the positive weight
    try:
      numerator += -neg_w * deviations[j][m]
      denominator += abs(neg_w)
    except KeyError:
      # neighbor may not have rated the same movie
      # don't want to do dictionary lookup twice
      # so just throw exception
      pass

  if denominator == 0:
    prediction = averages[i]
  else:
    prediction = numerator / denominator + averages[i]
  prediction = min(5, prediction)
  prediction = max(0.5, prediction) # min rating is 0.5
  return prediction

In [11]:

train_predictions = []
train_targets = []
for (i, m), target in usermovie2rating.items():
  # calculate the prediction for this movie
  prediction = predict(i, m)

  # save the prediction and target
  train_predictions.append(prediction)
  train_targets.append(target)

test_predictions = []
test_targets = []
# same thing for test set
for i in range(N):
    print(f"Processing user {i}")
    try:
        for neg_w, j in neighbors[i]:
            # Your existing code to compute prediction
            prediction = predict(i, m)
            # Your existing code to save predictions
            train_predictions.append(prediction)
            train_targets.append(target)
    except IndexError:
        print(f"IndexError: neighbors[{i}] is out of range")
        continue


IndexError: list index out of range

In [10]:
# calculate accuracy
def mse(p, t):
  p = np.array(p)
  t = np.array(t)
  return np.mean((p - t)**2)

print('train mse:', mse(train_predictions, train_targets))
print('test mse:', mse(test_predictions, test_targets))

train mse: nan


/opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


NameError: name 'test_predictions' is not defined